In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#!pip install keras --upgrade

!pip install -U keras

# !pip install tensorflow==1.14.0  # After running this, I get some instruction in red color which is
                                  # 'RESTART KERNEL' button below red color instr. . Then click on
                                  # that button.  



     |████████████████████████████████| 378kB 17.0MB/s 
  Found existing installation: Keras 2.2.5
    Uninstalling Keras-2.2.5:
      Successfully uninstalled Keras-2.2.5


In [0]:
import pandas as pd
import numpy as np 
import keras 
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.mobilenet import MobileNet
from keras.applications.resnet import ResNet101
#from keras.applications import ResNet50



from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input, decode_predictions
from keras.layers import Dense, Activation, Flatten, merge,Input
from keras.models import Model, Sequential
from keras.utils import np_utils
from keras import backend as K 
from keras.layers import *
from keras.callbacks import *
import os
import glob
import tensorflow as tf 

from keras.preprocessing.image import ImageDataGenerator, load_img 

import warnings
warnings.filterwarnings("ignore") 

Using TensorFlow backend.


In [0]:
# train = pd.read_csv('/kaggle/input/image-classification-dataset-in-the-gala-event/image_auto_tagging/train.csv')
train = pd.read_csv('/content/drive/My Drive/Hackerearth/toons-detection/Train.csv')

# test  = pd.read_csv('/kaggle/input/image-classification-dataset-in-the-gala-event/image_auto_tagging/test.csv')
test  = pd.read_csv('/content/drive/My Drive/Hackerearth/toons-detection/Test.csv')

s = train['Emotion'].tolist() 
 
from collections import Counter

print(Counter(s).keys()) # equals to list(set(words))
print(Counter(s).values()) # counts the elements' frequency
print(len(Counter(s).keys()))

# tg_dict = {"Food":0, "misc": 1, "Attire": 2,"Decorationandsignage":3}

tg_dict = {"Unknown":0, "angry": 1, "happy": 2, "sad":3, "surprised":4} 

def label_encode(x):
    return tg_dict[x]

train['Emotion'] = train['Emotion'].apply(label_encode)

images = train['Frame_ID'].tolist()
classes = train['Emotion'].tolist()

features=[]
labels=[]
# path = '/kaggle/input/image-classification-dataset-in-the-gala-event/image_auto_tagging/Train_Images/'
# path = '/content/drive/My Drive/Hackerearth/dataset/Train Images/'

path = '/content/drive/My Drive/Hackerearth/toons-detection/train_frames/'

for i in range(0,298): 
  if os.path.isfile(path+str(images[i])):
    pic = image.load_img(path+str(images[i]), target_size=(224, 224))
    #print(path+str(images[i]))
    x = image.img_to_array(pic)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features.append(x)
    labels.append(classes[i])
  else:
    print(path+str(images[i]), 'not present')
    
npfeatures = np.array(features)
print(npfeatures.shape)
img_dt = np.rollaxis(npfeatures, 1, 0)
print(img_dt.shape)
X = img_dt[0]
print(X.shape)
labels = np.array(labels)
# Y = np_utils.to_categorical(labels,4)
Y = np_utils.to_categorical(labels,5)

print(Y.shape)


from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

dict_keys(['happy', 'surprised', 'angry', 'Unknown', 'sad'])
dict_values([37, 68, 37, 116, 40])
5
(298, 1, 224, 224, 3)
(1, 298, 224, 224, 3)
(298, 224, 224, 3)
(298, 5)


In [0]:
IMAGE_SIZE=[224,224]
pretrained_model = MobileNet(weights='imagenet', include_top=False ,input_shape=[*IMAGE_SIZE, 3])

# Freeze all the layers
for layer in pretrained_model.layers[:]:
    layer.trainable = False 

# Check the trainable status of the individual layers
for layer in pretrained_model.layers:
    print(layer, layer.trainable)


from keras import models
from keras import layers
from keras import optimizers

# Create the model
model = models.Sequential()

# Add the vgg convolutional base model
model.add(pretrained_model) 

# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(220, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(220, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(5, activation='softmax'))

# Show a summary of the model. Check the number of trainable parameters
model.summary()

17227776/17225924 [==============================] - 1s 0us/step
<keras.engine.input_layer.InputLayer object at 0x7fbee1f1c588> False
<keras.layers.convolutional.ZeroPadding2D object at 0x7fbee1f1ca58> False
<keras.layers.convolutional.Conv2D object at 0x7fbee1f1cba8> False
<keras.layers.normalization.BatchNormalization object at 0x7fbee1f1cc18> False
<keras.layers.advanced_activations.ReLU object at 0x7fbee1f1cd30> False
<keras.layers.convolutional.DepthwiseConv2D object at 0x7fbee14a3518> False
<keras.layers.normalization.BatchNormalization object at 0x7fbee14a3320> False
<keras.layers.advanced_activations.ReLU object at 0x7fbed0434dd8> False
<keras.layers.convolutional.Conv2D object at 0x7fbed03d8e80> False
<keras.layers.normalization.BatchNormalization object at 0x7fbed03e4470> False
<keras.layers.advanced_activations.ReLU object at 0x7fbed03f7e10> False
<keras.layers.convolutional.ZeroPadding2D object at 0x7fbed03d8ba8> False
<keras.layers.convolutional.DepthwiseConv2D object at 0

In [0]:
#model.compile(optimizer = optimizers.Adagrad(learning_rate=0.01), loss='categorical_crossentropy') 
# keras.optimizers.Adagrad(learning_rate=0.01)
model.compile(optimizer = optimizers.RMSprop(lr=1e-4) , loss='categorical_crossentropy', metrics=['accuracy']) 

model.fit(X, Y, batch_size=32, epochs=20, validation_split=.1)  

Train on 268 samples, validate on 30 samples
Epoch 1/20
268/268 [==============================] - 9s 33ms/step - loss: 4.0797 - val_loss: 0.7212
Epoch 2/20
268/268 [==============================] - 1s 3ms/step - loss: 2.3346 - val_loss: 0.6332
Epoch 3/20
268/268 [==============================] - 1s 3ms/step - loss: 1.7866 - val_loss: 0.7976
Epoch 4/20
268/268 [==============================] - 1s 3ms/step - loss: 1.3148 - val_loss: 0.6231
Epoch 5/20
268/268 [==============================] - 1s 3ms/step - loss: 1.0737 - val_loss: 0.7649
Epoch 6/20
268/268 [==============================] - 1s 3ms/step - loss: 1.0821 - val_loss: 0.5779
Epoch 7/20
268/268 [==============================] - 1s 3ms/step - loss: 0.9098 - val_loss: 0.5262
Epoch 8/20
268/268 [==============================] - 1s 3ms/step - loss: 0.9273 - val_loss: 0.7467
Epoch 9/20
268/268 [==============================] - 1s 3ms/step - loss: 0.7798 - val_loss: 0.6550
Epoch 10/20
268/268 [==============================] -

In [0]:
images_test = test['Frame_ID'].tolist()
test_features=[] 
# path_test = '/kaggle/input/image-classification-dataset-in-the-gala-event/image_auto_tagging/Test_Images/'
path_test = '/content/drive/My Drive/Hackerearth/toons-detection/test_frames/' 

for i in range(0,186): 
  if os.path.isfile(path_test+str(images_test[i])):
    pic = image.load_img(path_test+str(images_test[i]), target_size=(224, 224))
    #print(path+str(images[i]))
    x = image.img_to_array(pic)  
    x = np.expand_dims(x, axis=0) 
    x = preprocess_input(x)
    test_features.append(x)
  else:
    print(path_test+str(images[i]), 'not present')

In [0]:
test_features = np.array(test_features) 
print(test_features.shape)
test_features = np.rollaxis(test_features, 1, 0)
print(test_features.shape)
X_test = test_features[0]
print(X_test.shape)

(186, 1, 224, 224, 3)
(1, 186, 224, 224, 3)
(186, 224, 224, 3)


In [0]:
preds = model.predict(X_test) 

In [0]:
predictions=[] 
for i in preds:
    predictions.append(np.argmax(i))

In [0]:
test['Emotion'] = predictions 

In [0]:
gt_dict = dict((v,k) for k,v in tg_dict.items())

def inverse_encode(x):
    return gt_dict[x]

test['Emotion'] = test['Emotion'].apply(inverse_encode)

In [0]:
test.head(1)

,Frame_ID,Emotion
0,test0.jpg,angry


In [0]:
test.to_csv('/content/drive/My Drive/Hackerearth/toons-detection/Submission15.csv',header=True,index = None) 